In [39]:
%pip install -q datasets jiwer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import torch
import pandas as pd
import requests
from PIL import Image
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from torch.utils.data import Dataset, DataLoader
from datasets import load_metric
from tqdm.notebook import tqdm

# Path to the IAM image data
root_dir = '/Users/adamking/Documents/NaturalLanguageProcessing/nlp-final-project/IAM/image'

# Path to the gt_test.txt test information file
gt_file_path = os.path.join(root_dir, '../gt_test.txt')

# Read gt_test.txt and create file_names and texts lists
file_names = []
texts = []

with open(gt_file_path, 'r') as f:
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) == 2:
            file_names.append(parts[0])
            texts.append(parts[1])

# Check if any files are missing texts
assert len(file_names) == len(
    texts), "File names and texts lists must be of the same length"

In [4]:
# Load the best model before final evaluation
device = torch.device('cpu')
model = VisionEncoderDecoderModel.from_pretrained(
    'microsoft/trocr-base-handwritten')
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')

# Set the decoder_start_token_id and pad_token_id
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Define the IAM dataset class
class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text
        file_name = self.df['file_name'][idx]
        text = self.df['text'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(os.path.join(
            self.root_dir, file_name)).convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(
            text, padding="max_length", max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label !=
                  self.processor.tokenizer.pad_token_id else -100 for label in labels]
        encoding = {"pixel_values": pixel_values.squeeze(),
                    "labels": torch.tensor(labels)}
        return encoding

In [5]:
# Create DataFrame
df = pd.DataFrame({'file_name': file_names, 'text': texts})

# Split DataFrame into train, test, and validation sets
train_df = df.sample(frac=0.8, random_state=42).reset_index(drop=True)
remaining = df.drop(train_df.index).reset_index(drop=True)
test_df = remaining.sample(frac=0.5, random_state=42).reset_index(drop=True)
valid_df = remaining.drop(test_df.index).reset_index(drop=True)

# Create datasets
train_dataset = IAMDataset(root_dir=root_dir, df=train_df, processor=processor)
test_dataset = IAMDataset(root_dir=root_dir, df=test_df, processor=processor)
eval_dataset = IAMDataset(root_dir=root_dir, df=valid_df, processor=processor)

# DataLoaders
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(eval_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [8]:
def train_epoch(model, dataloader, optimizer, device):
    """
    Trains the model for one epoch.

    Args:
        model: The model to be trained. This should be a neural network model compatible with the data.
        dataloader: DataLoader providing batches of data to train the model. Each batch should contain 'pixel_values' and 'labels'.
        optimizer: The optimizer used to update the model's parameters.
        device: The device on which to perform training (e.g., 'cpu' or 'cuda').

    Returns:
        float: The average loss over the epoch, computed as the total loss divided by the number of batches.
    """
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc="Training"):
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)

In [6]:
# Define the early stopping class for training
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

In [10]:
def evaluate(model, dataloader, device):
    """
    Evaluates the model on the given dataset.

    Args:
        model: The model to be evaluated. This should be a neural network model compatible with the data.
        dataloader: DataLoader providing batches of data for evaluation. Each batch should contain 'pixel_values' and 'labels'.
        device: The device on which to perform the evaluation (e.g., 'cpu' or 'cuda').

    Returns:
        float: The average loss over the evaluation dataset, computed as the total loss divided by the number of batches.
    """
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            pixel_values = batch["pixel_values"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(pixel_values=pixel_values, labels=labels)
            loss = outputs.loss

            total_loss += loss.item()
    return total_loss / len(dataloader)


# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
num_epochs = 7  # Increase this, as early stopping will prevent overfitting
early_stopping = EarlyStopping(patience=5, min_delta=0.01)

In [11]:
# Training loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    train_loss = train_epoch(model, train_loader, optimizer, device)
    print(f"Training loss: {train_loss}")

    val_loss = evaluate(model, valid_loader, device)
    print(f"Validation loss: {val_loss}")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early stopping")
        break

    # Optional: Save the model if it's the best so far
    if early_stopping.best_loss == val_loss:
        torch.save(model.state_dict(), 'best_model.pth')

# Load the best model before final evaluation
model.load_state_dict(torch.load('best_model.pth'))

# Final evaluation on test set
test_loss = evaluate(model, test_loader, device)
print(f"Test loss: {test_loss}")

Epoch 1/7


Training:   0%|          | 0/292 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [7]:
import torch
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from datasets import load_metric
from tqdm.notebook import tqdm

# Function to calculate accuracy
def calculate_accuracy(pred_str, label_str):
    """
    Calculates the accuracy of model predictions by comparing them to the true labels.

    Args:
        pred_str (list of str): A list of predicted strings from the model.
        label_str (list of str): A list of true label strings to compare against the predictions.

    Returns:
        float: The accuracy of the predictions, calculated as the ratio of correct predictions to the total number of predictions.
    """
    correct = 0
    total = 0
    for pred, label in zip(pred_str, label_str):
        # Add threshold for 'correct' for accuracy
        if pred.strip() == label.strip():
            correct += 1
        total += 1
    accuracy = correct / total if total > 0 else 0
    return accuracy

# Function to evaluate the model and calculate accuracy and CER
def evaluate_model(model, dataloader, processor, device):
    """
    Evaluates the model on a given dataset, calculating both the average loss and accuracy, as well as the Character Error Rate (CER).

    Args:
        model: The model to be evaluated. It should be a neural network model compatible with the data provided by the dataloader.
        dataloader: DataLoader providing batches of data for evaluation. Each batch should contain 'pixel_values' and 'labels'.
        processor: A processor used to decode the model's predictions and labels. Typically, this is a tokenizer or similar processing tool.
        device: The device on which to perform the evaluation (e.g., 'cpu' or 'cuda').

    Returns:
        tuple: A tuple containing:
            - float: The average loss over the evaluation dataset.
            - float: The accuracy of the model's predictions.
            - float: The Character Error Rate (CER), a metric used to measure the performance of text recognition models.
    """
    model.eval()
    total_loss = 0
    all_pred_str = []
    all_label_str = []

    # Character error rate for accuracy measurement
    cer_metric = load_metric("cer")

    with torch.no_grad():
        for batch in tqdm(dataloader):
            pixel_values = batch["pixel_values"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(pixel_values=pixel_values, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            generated_ids = model.generate(pixel_values)
            pred_str = processor.batch_decode(
                generated_ids, skip_special_tokens=True)
            labels[labels == -100] = processor.tokenizer.pad_token_id
            label_str = processor.batch_decode(
                labels, skip_special_tokens=True)

            all_pred_str.extend(pred_str)
            all_label_str.extend(label_str)
            cer_metric.add_batch(predictions=pred_str, references=label_str)

    avg_loss = total_loss / len(dataloader)
    accuracy = calculate_accuracy(all_pred_str, all_label_str)
    cer = cer_metric.compute()

    return avg_loss, accuracy, cer

# Load the saved model state
model.load_state_dict(torch.load(
    '/Users/adamking/Documents/NaturalLanguageProcessing/best_model.pth', map_location=device))
model.to(device)

# Final evaluation on test set
test_loss, test_accuracy, test_cer = evaluate_model(
    model, test_loader, processor, device)
print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_accuracy * 100:.2f}%")
print(f"Character error rate on test set: {test_cer * 100:.2f}%")

/var/folders/mz/rf2cw1qx0fl_1dc6x5hbt0300000gn/T/ipykernel_16278/1515564877.py:26: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  cer_metric = load_metric("cer")


  0%|          | 0/37 [00:00<?, ?it/s]

/Users/adamking/anaconda3/envs/nlp/lib/python3.10/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Test loss: 0.526094084957967
Test accuracy: 49.66%
Character error rate on test set: 10.57%


In [9]:
# Test specific example
url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSoolxi9yWGAT5SLZShv8vVd0bz47UWRzQC19fDTeE8GmGv_Rn-PCF1pP1rrUx8kOjA4gg&usqp=CAU'
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

pixel_values = processor(images=image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(
    generated_ids, skip_special_tokens=True)[0]
print("Generated text:", generated_text)

Generated text: Hope you have alone it.
